In [1]:
# 0. Load drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. Install dependency

!pip install -q git+https://github.com/dccastro/Morpho-MNIST.git
!pip install -q causalpfn torch torchvision tqdm pandas scikit-learn
!pip install -q timm

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 61.1 MB/s eta 0:00:00


In [3]:
# 2. Import packages

import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import json

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torchvision import datasets, transforms

from morphomnist import measure
from causalpfn import CATEEstimator, ATEEstimator

import timm
from torchvision.transforms import InterpolationMode

from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, LogisticRegression

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [4]:
# 3. Load Dino & MNIST

# Dino
dino_model_name = "vit_small_patch16_224.dino"

dino = timm.create_model(
    dino_model_name,
    pretrained=True,
)

dino.reset_classifier(0)
dino = dino.to(device)
dino.eval()

print("Loaded DINO model:", dino_model_name)

# MNIST
metrics_path = "/content/drive/MyDrive/morpho_metrics_train.csv"

transform = transforms.Compose([
    transforms.ToTensor(),
])

mnist_train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)

images = mnist_train.data.numpy().astype(np.uint8)          # (60000, 28, 28)
labels = mnist_train.targets.numpy().astype(np.int64)       # (60000,)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

if os.path.exists(metrics_path):
    print("Reading Morpho-MNIST metrics from drive：", metrics_path)
    metrics_df = pd.read_csv(metrics_path)
else:
    print("No downloaded metrics，start calculating Morpho-MNIST metrics ...")
    metrics_df = measure.measure_batch(images)
    metrics_df.to_csv(metrics_path, index=False)
    print("Finish calculation, save to：", metrics_path)

print(metrics_df.head())

thickness = metrics_df["thickness"].to_numpy().astype(np.float32)
slant     = metrics_df["slant"].to_numpy().astype(np.float32)

intensity = images.reshape(len(images), -1).mean(axis=1).astype(np.float32)

print("thickness shape:", thickness.shape)
print("slant shape:", slant.shape)
print("intensity shape:", intensity.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Loaded DINO model: vit_small_patch16_224.dino


100%|██████████| 9.91M/9.91M [00:00<00:00, 15.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 610kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.57MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.50MB/s]


Images shape: (60000, 28, 28)
Labels shape: (60000,)
Reading Morpho-MNIST metrics from drive： /content/drive/MyDrive/morpho_metrics_train.csv
       area     length  thickness     slant      width     height
0  107.1875  47.748737   2.525400  0.231577  14.523651  19.889152
1  123.0625  52.905592   2.570187  0.330892  15.385168  19.359169
2   78.1875  47.109650   1.853382 -0.213113  20.472912  19.133937
3   67.3750  23.202796   2.922822  0.507930   5.406406  19.768806
4   91.0000  46.223611   2.244745  0.008993  13.273560  19.492584
thickness shape: (60000,)
slant shape: (60000,)
intensity shape: (60000,)


In [5]:
# 4. Extract image embeddings

dino_embed_path = "/content/drive/MyDrive/mnist_dino_embeds.npy"

class MNISTForDINO(Dataset):
    def __init__(self, images_np):
        """
        images_np: (N, 28, 28) uint8
        """
        self.images = images_np
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224, interpolation=InterpolationMode.BICUBIC),
            transforms.Lambda(lambda img: img.convert("RGB")),
            transforms.ToTensor(),  # [0,1]
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std =[0.229, 0.224, 0.225],
            ),
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img = self.transform(img)
        return img

if os.path.exists(dino_embed_path):
    print("Loading extracted embeddings from:", dino_embed_path)
    dino_embs = np.load(dino_embed_path)
else:
    print("No downloaded DINO embedding, start extracting...")

    dataset_dino = MNISTForDINO(images)
    loader_dino = DataLoader(
        dataset_dino,
        batch_size=128,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    all_embs = []
    with torch.no_grad():
        for batch in tqdm(loader_dino, desc="Extracting DINO embeddings"):
            batch = batch.to(device)
            feats = dino(batch)          # (B, d)
            if isinstance(feats, (tuple, list)):
                feats = feats[0]
            all_embs.append(feats.cpu())

    dino_embs = torch.cat(all_embs, dim=0).numpy().astype(np.float32)
    print("DINO embeddings shape:", dino_embs.shape)

    np.save(dino_embed_path, dino_embs)
    print("Finish extracting DINO embeddings, save to:", dino_embed_path)

print("DINO embeddings shape:", dino_embs.shape)

Loading extracted embeddings from: /content/drive/MyDrive/mnist_dino_embeds.npy
DINO embeddings shape: (60000, 384)


In [6]:
# 5. Normalize + subsample

N_TOTAL = len(images)
N_SUBSAMPLE = 20000

rng = np.random.default_rng(42)
indices = rng.choice(N_TOTAL, size=N_SUBSAMPLE, replace=False)

digit     = labels[indices]
phi_thick = thickness[indices]
w_slant   = slant[indices]
inten     = intensity[indices]
dino_sub  = dino_embs[indices]

def zscore(x):
    return (x - x.mean()) / (x.std() + 1e-8)

z_thick = zscore(phi_thick)
z_slant = zscore(w_slant)
z_inten = zscore(inten)

print("Subsampled N:", len(digit))
print("dino_sub shape:", dino_sub.shape)

Subsampled N: 20000
dino_sub shape: (20000, 384)


In [7]:
# 6. Define World A / World B

def simulate_world_A(z_thick, z_slant, seed=0):
    """
    World A: only thickness is confounder,
    T, Y only depends on z_thick。
    """
    rng = np.random.default_rng(seed)
    n = len(z_thick)

    gamma0, gamma1 = 0.0, 1.0
    logits_T = gamma0 + gamma1 * z_thick
    p_T = 1 / (1 + np.exp(-logits_T))
    T = rng.binomial(1, p_T).astype(np.float32)

    beta0, beta1 = 0.0, 1.0
    tau0, tau1   = 0.5, 0.5
    eps = rng.normal(0.0, 1.0, size=n).astype(np.float32)

    Y0 = beta0 + beta1 * z_thick + eps
    tau_true = tau0 + tau1 * z_thick
    Y1 = Y0 + tau_true

    Y = Y0 * (1 - T) + Y1 * T

    return {
        "T": T,
        "Y": Y.astype(np.float32),
        "Y0": Y0.astype(np.float32),
        "Y1": Y1.astype(np.float32),
        "tau": tau_true.astype(np.float32),
    }


def simulate_world_B(z_thick, z_slant, seed=0):
    """
    World B: thickness and slant are confounders。
    - T depends on thickness + slant
    - Y0, τ(x) depends on thickness + slant
    """
    rng = np.random.default_rng(seed)
    n = len(z_thick)

    gamma0, gamma1, gamma2 = 0.0, 0.8, 1.2
    logits_T = gamma0 + gamma1 * z_thick + gamma2 * z_slant
    p_T = 1 / (1 + np.exp(-logits_T))
    T = rng.binomial(1, p_T).astype(np.float32)

    beta0, beta1, beta2 = 0.0, 1.0, 1.0
    tau0, tau1, tau2    = 0.5, 0.5, 0.5
    eps = rng.normal(0.0, 1.0, size=n).astype(np.float32)

    Y0 = beta0 + beta1 * z_thick + beta2 * z_slant + eps
    tau_true = tau0 + tau1 * z_thick + tau2 * z_slant
    Y1 = Y0 + tau_true

    Y = Y0 * (1 - T) + Y1 * T

    return {
        "T": T,
        "Y": Y.astype(np.float32),
        "Y0": Y0.astype(np.float32),
        "Y1": Y1.astype(np.float32),
        "tau": tau_true.astype(np.float32),
    }


In [8]:
# 7. Define input & World data

X_tab = z_thick.reshape(-1, 1).astype(np.float32)
X_img_dino = dino_sub.astype(np.float32)
X_tab_img_dino = np.concatenate(
    [z_thick.reshape(-1, 1).astype(np.float32), X_img_dino],
    axis=1
)

worldA = simulate_world_A(z_thick, z_slant, seed=0)
worldB = simulate_world_B(z_thick, z_slant, seed=1)

T_A   = worldA["T"]
Y_A   = worldA["Y"]
tau_A = worldA["tau"]

T_B   = worldB["T"]
Y_B   = worldB["Y"]
tau_B = worldB["tau"]

ids_A = np.arange(len(T_A), dtype=int)
ids_B = np.arange(len(T_B), dtype=int)

print("N_A:", len(T_A), "N_B:", len(T_B))
print("X_tab shape:", X_tab.shape)
print("X_img_dino shape:", X_img_dino.shape)
print("X_tab_img_dino shape:", X_tab_img_dino.shape)

N_A: 20000 N_B: 20000
X_tab shape: (20000, 1)
X_img_dino shape: (20000, 384)
X_tab_img_dino shape: (20000, 385)


In [9]:
# 8. Sanity check

def sanity_check_world(world_data, z_thick, z_slant, name="A"):
    """
    Do sanity check for world (A/B):
    1) Check relationship between T, Y, tau_true and thickness/slant
    2) Check if slant is irrelevent in World A, and relevent in World B
    3) Check overlap: logistic regression of propensity
    """
    T = world_data["T"].astype(float)
    Y = world_data["Y"].astype(float)
    tau = world_data["tau"].astype(float)

    print(f"\n====== Sanity check for World {name} ======")
    print("N =", len(T))

    df_tmp = pd.DataFrame({
        "thick": z_thick,
        "slant": z_slant,
        "T": T,
        "Y": Y,
        "tau": tau,
    })
    print("\n[World", name, "] Correlation matrix:")
    print(df_tmp.corr().round(3))

    X_2d = np.stack([z_thick, z_slant], axis=1)
    lin_Y = LinearRegression().fit(X_2d, Y)
    Y_pred = lin_Y.predict(X_2d)
    r2_Y = r2_score(Y, Y_pred)

    print(f"\n[World {name}] LinearRegression: Y ~ thick + slant")
    print("  coef_thick =", lin_Y.coef_[0], "  coef_slant =", lin_Y.coef_[1])
    print("  R^2 =", r2_Y)

    lin_tau = LinearRegression().fit(X_2d, tau)
    tau_pred = lin_tau.predict(X_2d)
    r2_tau = r2_score(tau, tau_pred)

    print(f"\n[World {name}] LinearRegression: tau ~ thick + slant")
    print("  coef_thick =", lin_tau.coef_[0], "  coef_slant =", lin_tau.coef_[1])
    print("  R^2 =", r2_tau)

    logit = LogisticRegression(max_iter=1000).fit(X_2d, T)
    ps = logit.predict_proba(X_2d)[:, 1]

    print(f"\n[World {name}] LogisticRegression: T ~ thick + slant")
    print("  coef_thick =", logit.coef_[0][0], "  coef_slant =", logit.coef_[0][1], "  intercept =", logit.intercept_[0])
    print("  Propensity range: min =", ps.min(), " max =", ps.max())

    print("\n====== End sanity check for World", name, "======\n")


sanity_check_world(worldA, z_thick, z_slant, name="A")
sanity_check_world(worldB, z_thick, z_slant, name="B")



====== Sanity check for World A ======
N = 20000

[World A ] Correlation matrix:
       thick  slant      T      Y    tau
thick  1.000  0.020  0.385  0.803  1.000
slant  0.020  1.000  0.011  0.010  0.020
T      0.385  0.011  1.000  0.412  0.385
Y      0.803  0.010  0.412  1.000  0.803
tau    1.000  0.020  0.385  0.803  1.000

[World A] LinearRegression: Y ~ thick + slant
  coef_thick = 1.415114   coef_slant = -0.01030814
  R^2 = 0.6453971508026426

[World A] LinearRegression: tau ~ thick + slant
  coef_thick = 0.50000006   coef_slant = 2.110571e-08
  R^2 = 0.9999999999999801

[World A] LogisticRegression: T ~ thick + slant
  coef_thick = 0.9775317621602353   coef_slant = 0.0006916068906344529   intercept = 0.010205897903427642
  Propensity range: min = 0.0976339602633423  max = 0.99994712380043

====== End sanity check for World A ======


====== Sanity check for World B ======
N = 20000

[World B ] Correlation matrix:
       thick  slant      T      Y    tau
thick  1.000  0.020  0.28

In [10]:
# 9. Linear probe: DINO_emb → thickness / slant

from sklearn.linear_model import LinearRegression

def linear_probe_feature(X_emb, target, name="slant"):
    reg = LinearRegression().fit(X_emb, target)
    pred = reg.predict(X_emb)
    r2 = r2_score(target, pred)
    print(f"[Linear probe] R^2(DINO_emb → {name}) = {r2:.4f}")
    return r2

print("\n========== Linear probe on DINO embeddings ==========\n")

r2_slant = linear_probe_feature(X_img_dino, z_slant, name="slant")
r2_thick = linear_probe_feature(X_img_dino, z_thick, name="thickness")



========== Linear probe on DINO embeddings ==========

[Linear probe] R^2(DINO_emb → slant) = 0.6582
[Linear probe] R^2(DINO_emb → thickness) = 0.9707


In [11]:
# 10. Define metrics and pipeline

def rmse(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    return float(np.sqrt(np.mean((a - b)**2)))

def policy_metrics(tau_hat, tau_true, ks=(0.1, 0.2, 0.3)):
    out = []
    n = len(tau_hat)
    order_hat  = np.argsort(-tau_hat)
    order_true = np.argsort(-tau_true)
    for k in ks:
        m = max(1, int(round(k * n)))
        gh = float(np.mean(tau_true[order_hat[:m]]))
        go = float(np.mean(tau_true[order_true[:m]]))
        regret = 1.0 - gh / (go + 1e-12)
        out.append((k, gh, go, regret))
    return out


def eval_causalpfn_morpho(
    Z,                 # input
    T_all,             # treatment
    Y_all,             # outcome
    tau_true_all,      # true ITE
    name="DINO_only",  # Pipeline name
    K=5,               # K-fold
    pca_dim=None,
    ids=None
):
    """
    CausalPFN evaluation function：
    - Use KFold；
    - Output CATE_R2、PEHE、ATE_hat、ATE_true、ATE_MAE；
    - Output policy metrics（@10%, 20%, 30%）；
    - Use ATEEstimator to get ATE estimation；
    - Save cate_pred_xxx.csv and run_config_xxx.json。
    """

    Z = np.asarray(Z, np.float32)
    T_all = np.asarray(T_all, np.float32)
    Y_all = np.asarray(Y_all, np.float32)
    tau_true_all = np.asarray(tau_true_all, np.float32)

    N, d = Z.shape
    if ids is None:
        ids = np.arange(N)

    cate_pred = np.zeros(N, dtype=np.float32)

    kf = KFold(n_splits=K, shuffle=True, random_state=0)

    for fold, (tr, va) in enumerate(kf.split(Z), start=1):
        Z_tr, Z_va = Z[tr], Z[va]
        T_tr, Y_tr = T_all[tr], Y_all[tr]
        tau_va     = tau_true_all[va]

        if pca_dim is not None and pca_dim < Z_tr.shape[1]:
            pca = PCA(n_components=pca_dim, random_state=0).fit(Z_tr)
            Ze_tr = pca.transform(Z_tr)
            Ze_va = pca.transform(Z_va)
        else:
            Ze_tr, Ze_va = Z_tr, Z_va

        sc = StandardScaler().fit(Ze_tr)
        X_tr = sc.transform(Ze_tr).astype(np.float32)
        X_va = sc.transform(Ze_va).astype(np.float32)

        cate_est = CATEEstimator(device=device, verbose=False)
        cate_est.fit(X_tr, T_tr, Y_tr)
        cate_pred[va] = cate_est.estimate_cate(X_va).astype(np.float32)

        print(f"[{name}][Fold {fold}] Train={len(tr)}  Val={len(va)}  done.")

    cate_r2   = r2_score(tau_true_all, cate_pred)
    pehe_rmse = rmse(cate_pred, tau_true_all)
    ate_hat   = float(np.mean(cate_pred))
    ate_true  = float(np.mean(tau_true_all))
    ate_mae   = abs(ate_hat - ate_true)
    pol       = policy_metrics(cate_pred, tau_true_all, ks=(0.10, 0.20, 0.30))

    print(f"[{name}] CATE_R2={cate_r2:.4f}  PEHE={pehe_rmse:.4f}  "
          f"ATE_hat={ate_hat:.4f}  ATE_true={ate_true:.4f}  ATE_MAE={ate_mae:.4f}")
    for k,gh,go,rg in pol:
        print(f"[{name}][Policy@{int(k*100)}%] gain_hat={gh:.4f}  gain_oracle={go:.4f}  norm_regret={rg:.4f}")

    ate_est = ATEEstimator(device=device, verbose=False)
    ate_est.fit(Z.astype(np.float32), T_all, Y_all)
    ate_hat_global = float(ate_est.estimate_ate())
    print(f"[{name}][ATEEstimator] ATE_hat_global={ate_hat_global:.4f}")

    os.makedirs("outputs", exist_ok=True)
    out_df = pd.DataFrame({
        "id": ids,
        "tau_hat": cate_pred,
        "tau_true": tau_true_all,
        "T": T_all,
        "Y": Y_all,
    })
    out_path = f"outputs/cate_pred_{name}.csv"
    out_df.to_csv(out_path, index=False)

    run_cfg = {
        "name": name,
        "n": int(N),
        "d_in": int(d),
        "K": int(K),
        "pca_dim": int(pca_dim) if pca_dim is not None else None,
        "metrics": {
            "CATE_R2": float(cate_r2),
            "PEHE_RMSE": float(pehe_rmse),
            "ATE_hat": float(ate_hat),
            "ATE_true": float(ate_true),
            "ATE_MAE": float(ate_mae),
            "Policy": [
                {"k": float(k), "gain_hat": float(gh), "gain_oracle": float(go), "norm_regret": float(rg)}
                for (k,gh,go,rg) in pol
            ],
            "ATEEstimator_global": float(ate_hat_global)
        }
    }
    cfg_path = f"outputs/run_config_{name}.json"
    json.dump(run_cfg, open(cfg_path, "w"), indent=2)
    print("Saved:", out_path, "and", cfg_path)

    return cate_pred, run_cfg

In [12]:
# 11. Oracle baseline: Z = [thickness, slant]

X_oracle = np.stack([z_thick, z_slant], axis=1).astype(np.float32)
ids = np.arange(len(z_thick))

print("\n========== Oracle PFN with [thick, slant] ==========\n")

# World A
cate_A_oracle, cfg_A_oracle = eval_causalpfn_morpho(
    Z=X_oracle,
    T_all=worldA["T"],
    Y_all=worldA["Y"],
    tau_true_all=worldA["tau"],
    name="WorldA_oracle_thick_slant",
    K=5,
    pca_dim=None,
    ids=ids,
)

# World B
cate_B_oracle, cfg_B_oracle = eval_causalpfn_morpho(
    Z=X_oracle,
    T_all=worldB["T"],
    Y_all=worldB["Y"],
    tau_true_all=worldB["tau"],
    name="WorldB_oracle_thick_slant",
    K=5,
    pca_dim=None,
    ids=ids,
)



========== Oracle PFN with [thick, slant] ==========



causalpfn_v0.pt:   0%|          | 0.00/75.4M [00:00<?, ?B/s]

[WorldA_oracle_thick_slant][Fold 1] Train=16000  Val=4000  done.
[WorldA_oracle_thick_slant][Fold 2] Train=16000  Val=4000  done.
[WorldA_oracle_thick_slant][Fold 3] Train=16000  Val=4000  done.
[WorldA_oracle_thick_slant][Fold 4] Train=16000  Val=4000  done.
[WorldA_oracle_thick_slant][Fold 5] Train=16000  Val=4000  done.
[WorldA_oracle_thick_slant] CATE_R2=0.9301  PEHE=0.1322  ATE_hat=0.5108  ATE_true=0.5000  ATE_MAE=0.0108
[WorldA_oracle_thick_slant][Policy@10%] gain_hat=1.5270  gain_oracle=1.5352  norm_regret=0.0053
[WorldA_oracle_thick_slant][Policy@20%] gain_hat=1.2420  gain_oracle=1.2651  norm_regret=0.0183
[WorldA_oracle_thick_slant][Policy@30%] gain_hat=1.0679  gain_oracle=1.1001  norm_regret=0.0293
[WorldA_oracle_thick_slant][ATEEstimator] ATE_hat_global=0.4996
Saved: outputs/cate_pred_WorldA_oracle_thick_slant.csv and outputs/run_config_WorldA_oracle_thick_slant.json
[WorldB_oracle_thick_slant][Fold 1] Train=16000  Val=4000  done.
[WorldB_oracle_thick_slant][Fold 2] Train=16

In [13]:
# 12. Fully evaluate World A / World B

# World A：Image is redundant
print("\n========== World A: Image is redundant ==========\n")

cate_A_tab, cfg_A_tab = eval_causalpfn_morpho(
    Z=X_tab,              # (N, 1)
    T_all=T_A,
    Y_all=Y_A,
    tau_true_all=tau_A,
    name="WorldA_tab_only",
    K=5,
    pca_dim=None,
    ids=ids_A,
)

cate_A_img, cfg_A_img = eval_causalpfn_morpho(
    Z=X_img_dino,
    T_all=T_A,
    Y_all=Y_A,
    tau_true_all=tau_A,
    name="WorldA_dino_only",
    K=5,
    pca_dim=48,
    ids=ids_A,
)

cate_A_tab_img, cfg_A_tab_img = eval_causalpfn_morpho(
    Z=X_tab_img_dino,
    T_all=T_A,
    Y_all=Y_A,
    tau_true_all=tau_A,
    name="WorldA_tab_plus_dino",
    K=5,
    pca_dim=48,
    ids=ids_A,
)


#World B：Image has hidden confounder
print("\n========== World B: Image has hidden confounder ==========\n")

cate_B_tab, cfg_B_tab = eval_causalpfn_morpho(
    Z=X_tab,
    T_all=T_B,
    Y_all=Y_B,
    tau_true_all=tau_B,
    name="WorldB_tab_only",
    K=5,
    pca_dim=None,
    ids=ids_B,
)

cate_B_img, cfg_B_img = eval_causalpfn_morpho(
    Z=X_img_dino,
    T_all=T_B,
    Y_all=Y_B,
    tau_true_all=tau_B,
    name="WorldB_dino_only",
    K=5,
    pca_dim=48,
    ids=ids_B,
)

cate_B_tab_img, cfg_B_tab_img = eval_causalpfn_morpho(
    Z=X_tab_img_dino,
    T_all=T_B,
    Y_all=Y_B,
    tau_true_all=tau_B,
    name="WorldB_tab_plus_dino",
    K=5,
    pca_dim=48,
    ids=ids_B,
)



========== World A: Image is redundant ==========

[WorldA_tab_only][Fold 1] Train=16000  Val=4000  done.
[WorldA_tab_only][Fold 2] Train=16000  Val=4000  done.
[WorldA_tab_only][Fold 3] Train=16000  Val=4000  done.
[WorldA_tab_only][Fold 4] Train=16000  Val=4000  done.
[WorldA_tab_only][Fold 5] Train=16000  Val=4000  done.
[WorldA_tab_only] CATE_R2=0.9681  PEHE=0.0893  ATE_hat=0.5120  ATE_true=0.5000  ATE_MAE=0.0120
[WorldA_tab_only][Policy@10%] gain_hat=1.5338  gain_oracle=1.5352  norm_regret=0.0009
[WorldA_tab_only][Policy@20%] gain_hat=1.2470  gain_oracle=1.2651  norm_regret=0.0143
[WorldA_tab_only][Policy@30%] gain_hat=1.0855  gain_oracle=1.1001  norm_regret=0.0133
[WorldA_tab_only][ATEEstimator] ATE_hat_global=0.5088
Saved: outputs/cate_pred_WorldA_tab_only.csv and outputs/run_config_WorldA_tab_only.json
[WorldA_dino_only][Fold 1] Train=16000  Val=4000  done.
[WorldA_dino_only][Fold 2] Train=16000  Val=4000  done.
[WorldA_dino_only][Fold 3] Train=16000  Val=4000  done.
[WorldA_d